# Chess Model Deployment with vLLM on Trainium

## Overview

This notebook demonstrates deploying a fine-tuned chess model using vLLM on AWS Trainium. You'll learn how to:

- Set up vLLM with Neuron backend for Trainium
- Deploy a compiled chess model with batch support
- Test the deployed model with interactive games
- Validate performance and understand concurrency benefits

**Prerequisites:**
- Complete the fine-tuning lab or use a pre-trained chess model
- Trainium instance (trn1.2xlarge or larger)

**Duration:** 30-40 minutes (including model compilation)

## Step 1: Install Dependencies

First, let's install the chess environment and its dependencies.

In [ ]:
%cd /home/ubuntu/environment/neuron-workshops/labs/vLLM/Chess/assets

# Install chess environment dependencies
!pip install -q -r requirements.txt

print("\n Dependencies installed successfully!")

In [ ]:
 # Upgrade to neuronx-cc 2.21.x
 !pip install --upgrade neuronx-cc==2.21.* --extra-index-url=https://pip.repos.neuron.amazonaws.com
 # Verify
 !neuronx-cc --version

## Step 3: Check Neuron Hardware

Verify Neuron cores are available and not in use.

In [ ]:
!neuron-ls

**Expected output:** You should see 2 Neuron cores available for trn1.2xlarge or 4 Neuron cores available for trn2.3xlarge
If cores are in use, you can free them with: `!pkill -f vllm` or `kill pid`

## Step 4: Configure Model Path

Set the path to your fine-tuned chess model. If you completed the fine-tuning lab, update the `FINETUNED_MODEL_PATH` below.

Otherwise, we'll use a pre-compiled model from the repository that we set with the HuggingFace `PRECOMPILED_MODEL_PATH`.

In [ ]:
import os

# Option 1: Use your fine-tuned model (update path if you completed fine-tuning lab)
FINE_TUNED_MODEL_PATH = "/home/ubuntu/environment/ml/qwen-chess/compiled_model" # default path from previous lab

# Option 2: Use pre-compiled model from HF repository (default), this will be pulled down by optimum-neuron-vllm
PRECOMPILED_MODEL_PATH = "kunhunjon/ChessLM_Qwen3_Trainium_AWS_Format"

# Verify fine-tuned model exists
if os.path.exists(FINE_TUNED_MODEL_PATH):
    print(f" Fine-tuned Model found at: {FINE_TUNED_MODEL_PATH}")
    
    # Check if fine-tuned model is compiled
    if os.path.exists(os.path.join(FINE_TUNED_MODEL_PATH, "model.pt")):
        print(" Fine-tuned model found (model.pt)")
        
        # Check model's neuron_config
        import json
        with open(os.path.join(FINE_TUNED_MODEL_PATH, "neuron_config.json")) as f:
            config = json.load(f)
        print(f"   Batch size: {config.get('batch_size', 'N/A')}")
        print(f"   Continuous batching: {config.get('continuous_batching', 'N/A')}")
        print(f"   Tensor parallel: {config.get('tp_degree', 'N/A')}")
    else:
        print("Model not compiled yet. Will compile on first run (10-30 min).")
else:
    print(f" Model not found at: {FINE_TUNED_MODEL_PATH}")
    print("Please update MODEL_PATH or complete the fine-tuning lab first.")

In [ ]:
# Based on the model you wish to move forward with in this lab, update the MODEL_PATH variable below
MODEL_PATH=PRECOMPILED_MODEL_PATH

## Step 5: Start vLLM Server

Now let's start the vLLM server with Neuron backend. The precompiled model is configured for:
- **batch_size=4**: Can process up to 4 requests concurrently
- **continuous_batching=true**: Automatically groups concurrent requests
- **tensor_parallel_size=2**: 

The fine-tuned model we just compiled is also configured for:
- **batch_size=4**: Can process up to 4 requests concurrently
- **continuous_batching=true**: Automatically groups concurrent requests
- **tensor_parallel_size=2**:  

**Note:** First-time startup includes model compilation which takes 10-30 minutes. Subsequent starts are fast (<1 min).

In [ ]:
# Update vllm.sh with correct model path
import os
import re

vllm_script = "vllm-server/vllm.sh"
with open(vllm_script, 'r') as f:
    content = f.read()

# Update MODEL_PATH in script using regex (works even if already modified)
content = re.sub(
    r'MODEL_PATH="[^"]*"',
    f'MODEL_PATH="{MODEL_PATH}"',
    content
)

with open(vllm_script, 'w') as f:
    f.write(content)

print(f" Updated vllm.sh with model path: {MODEL_PATH}")


In [ ]:
%%bash
cd vllm-server

# Kill existing server
pkill -f 'vllm.entrypoints.openai.api_server' || true
sleep 2

# Start in background with nohup
nohup bash vllm.sh > vllm-server.log 2>&1 &

echo "Server starting (PID: $!)"
echo "Check logs: tail -f vllm-server/vllm-server.log"


**While waiting for server startup:**

You can monitor the server logs in the terminal:
```bash
tail -f /home/ubuntu/environment/neuron-workshops/labs/vLLM/Chess/assets/vllm-server/vllm-server.log
```

Or check Neuron core usage:
```bash
watch -n 2 neuron-ls
```

## Step 6: Test Server Connection

Once the server has started, let's test the connection.

In [ ]:
import time
import openai

# Give server a moment to finish startup
print("Waiting for server to be fully ready...")
time.sleep(10)

# Initialize OpenAI client pointing to vLLM server
client = openai.OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="not-needed"  # vLLM doesn't require authentication
)

# Test connection
try:
    response = client.chat.completions.create(
        model=MODEL_PATH,
        messages=[{"role": "user", "content": "Hello"}],
        max_tokens=10,
        extra_body={"chat_template_kwargs": {"enable_thinking": False}}
    )
    print(" vLLM server is running!")
    print(f"\nTest response: {response.choices[0].message.content}")
except Exception as e:
    print(f" Connection failed: {e}")
    print("\nTroubleshooting:")
    print("1. Check if server is still starting (check logs)")
    print("2. Verify Neuron cores are not in use: neuron-ls")
    print("3. Check for errors: tail vllm-server/vllm-server.log")

## Step 8: Test Model Against Stockfish

Now let's test against a real chess engine to get a sense of the model's strength.

Note: prior to running any games, ensure your `.env` file is properly configured for vLLM and game parameters, or pass this to the agents directly.

In [ ]:
%cd /home/ubuntu/environment/neuron-workshops/labs/vLLM/Chess/

from assets.agents.vllm_agent import VLLMAgent
from assets.agents.stockfish_agent import StockfishAgent
from assets.env import ChessEnvironment

# Create Stockfish opponent (skill level 1 = beginner)
stockfish = StockfishAgent(skill_level=1, depth=2)

# Create vLLM agent
MODEL_PATH = "kunhunjon/ChessLM_Qwen3_Trainium_AWS_Format"
BASE_URL = "http://localhost:8080/v1"

# vLLM agent using the SAME config as your working test
vllm_agent = VLLMAgent(
    base_url=BASE_URL,
    model=MODEL_PATH,
    temperature=0.1,
    max_tokens=200,
    timeout=30.0,
    retry_attempts=1,  
    retry_delay=0.5,
)

# Create environment
env = ChessEnvironment(
    agent1=vllm_agent,
    agent2=stockfish,
    max_moves=100
)

# Play game
print("Playing chess game: vLLM vs Stockfish (skill=1)\n")
result = env.play_game(verbose=True)

print("\n" + "="*60)
print("Game Result")
print("="*60)
print(f"Result: {result['result']}")
print(f"Moves played: {result['moves_played']}")
print(f"Reason: {result['game_over_reason']}")
print(f"\nYour model's illegal move attempts: {result['white_illegal_attempts']}")

## Step 9: Validate Performance

Let's measure single-move latency to understand performance.

In [ ]:
import chess
import time

# Create a fresh board
board = chess.Board()
legal_moves = list(board.legal_moves)

# Measure latency over 10 moves
latencies = []
print("Measuring latency over 10 moves...\n")

for i in range(10):
    start = time.time()
    move, comment = vllm_agent.choose_move(
        board=board.copy(),
        legal_moves=legal_moves,
        move_history=[],
        side_to_move="White"
    )
    latency = time.time() - start
    latencies.append(latency)
    print(f"Move {i+1}: {latency:.3f}s")

# Calculate statistics
import statistics
print("\n" + "="*60)
print("Performance Metrics")
print("="*60)
print(f"Mean latency: {statistics.mean(latencies):.3f}s")
print(f"Median latency: {statistics.median(latencies):.3f}s")
print(f"Min latency: {min(latencies):.3f}s")
print(f"Max latency: {max(latencies):.3f}s")
print(f"Throughput: {1/statistics.mean(latencies):.2f} moves/second")

# Expected performance
expected_latency = 0.65  # seconds
if statistics.mean(latencies) < expected_latency * 1.2:
    print("\n Performance looks good!")
else:
    print(f"\n  Latency higher than expected (~{expected_latency}s). Check server configuration.")

## Understanding Concurrency Benefits

Your vLLM server is configured with:
- **`max_num_seqs=4`**: Can batch up to 4 requests
- **`continuous_batching=true`**: Automatically groups concurrent requests

### How It Works:

When you run multiple games in parallel (covered in Chess-Tournament.ipynb), the tournament scheduler uses Python multiprocessing to run games concurrently. Each game makes HTTP requests to the same vLLM server.

**With 1 game (sequential):**
- Latency: ~0.65s per move
- Throughput: 1.54 moves/second

**With 4 games (parallel):**
- Latency per game: ~1.86s per move (games wait in small batches)
- Total throughput: 2.15 moves/second across all games
- **Improvement: 1.4x faster** (games complete ~40% faster overall)

**Why the improvement?**
The vLLM server batches concurrent requests together, processing them simultaneously on the Neuron cores. This amortizes the overhead of model inference across multiple requests.

### Testing Concurrency:

You'll see this in action in the next notebook (Chess-Tournament.ipynb), where running with `--parallelism 4` enables automatic batching.


## Viewing Your Games on Chess.com Analysis

Once your games have been exported to PGN (e.g. `tournament_out/pgns/...` or `games.pgn`), you can replay and analyze them on Chess.com:

1. Open the `.pgn` file in a text editor (VS Code, Notepad, nano, etc.).
2. Select **one full game’s PGN** (from the `[Event "..."]` header down through the final result like `1-0`, `0-1`, or `1/2-1/2`).
3. Copy that text to your clipboard.
4. Go to [chess.com/analysis](https://www.chess.com/analysis).
5. Click **“Import Game”** / **“Import PGN”** (button name may vary slightly).
6. Paste the PGN into the text box.
7. Click **“Import”** / **“Analyze”**.

You’ll now see your game on an interactive board where you can:

- Step through each move
- Turn on engine analysis
- Check evaluation bars, blunders, and alternative lines


## Summary

Congratulations! You've successfully:

- Deployed a chess model via vLLM on Trainium  
- Configured vLLM for batching (max_num_seqs=4, continuous_batching)  
- Tested the model with interactive games  
- Validated performance metrics  
- Understood concurrency benefits (1.4x throughput improvement)

**Next Steps:**

Continue to [Chess-Tournament.ipynb](Chess-Tournament.ipynb) to:
- Run competitive tournaments with TrueSkill ratings
- Evaluate model strength against various Stockfish levels
- See automatic batching in action with parallel games
- Analyze detailed performance metrics

**Keep the vLLM server running** for the next notebook!